In [13]:
from glove import Corpus, Glove
import pandas as pd
import numpy as np

l1='1189'
l1='SCOPe_207'
loc = l1+'/'+l1+'.csv'
data = pd.read_csv(loc)

dim=400

data['features'] = data['Sequence']
#data['features'] = data['Seq-Code']
dataset=[]
for z in data['features']:
    dataset.append(list(z))
#print(dataset[0])
    

In [14]:
corpus = Corpus()

In [15]:
corpus.fit(dataset, window = 1)

In [16]:
glove = Glove(no_components = dim, learning_rate = 0.01)

In [17]:
glove.fit(matrix = corpus.matrix, epochs = 30, no_threads = 4, verbose = True)

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [18]:
glove.add_dictionary(corpus.dictionary)


In [19]:
glove.save('glove_model')

In [20]:
glove.load('glove_model')
#glove.most_similar('M', number = 50)

In [21]:
#print (len(glove.word_vectors[glove.dictionary['M']]))

def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        #print (word)
        try:
            #print(model[word])
            vec += glove.word_vectors[glove.dictionary[word]].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec
from sklearn.preprocessing import scale
feat_glove = np.concatenate([buildWordVector(z, dim) for z in data['features']])
feat_glove = scale(feat_glove)



In [22]:
def vector_converter(word):
    idx = glove.dictionary[word]
    return glove.word_vectors[idx]

In [23]:
def cosine_similarity(v1, v2):
    return 1 - spatial.distance.cosine(v1, v2)

In [24]:
df=pd.DataFrame(feat_glove)
df['PDBid'] = data['PDBid']
df['Class'] = data['Class']
df.to_csv(l1+'/CE_Seq_'+l1+'.csv')
#df.to_csv(l1+'/CE_Str_'+l1+'.csv')